# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera
### By: Matias Garib

This Jupyter Notebook contains all the code and brief comments of the Coursera Capstone project. The full report will be accessible in the following Github Repository: https://github.com/MatiasGarib/Coursera_Capstone

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Datasets](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)


## Introduction <a name="introduction"></a>

People want to start going out and visiting restaurants, but they want to visit places with the best hygiene practices. The questions we want to answer, for the city of San Francisco, are: which are the cleanest restaurants in each neighborhood? Which are the safest neighborhoods to go out to eat?

## Datasets <a name="data"></a>

1. The first dataset to be used consists of a **GeoJSON file with the names and boundaries of 41 San Francisco neighborhoods (GeoJSON)** 
2. **Foursquare APIs (URI)**
3. City of San Francisco Health Department’s **hygiene inspection program (CSV)** 


In [5]:
pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pandas as pd
import numpy as np
import requests
import folium
from sodapy import Socrata



<h3> Importinge the Datasets <h3>

The neighborhoods and hygiene inspection datasets are easily accessible thanks to the Socrata API provided by the San Francisco Government

In [101]:
client = Socrata("data.sfgov.org", None)
results = client.get("pyih-qa8i", limit=60000)
hygiene_df=pd.DataFrame.from_records(results)

nhoods=client.get("743h-p4bq", limit=60000) # We will use this JSON file later on to map out San Francisco's neighborhoods
nhoods_df=pd.DataFrame.from_records(nhoods)

In [102]:
print(hygiene_df.shape)
print(nhoods_df.shape)

(53973, 23)
(92, 4)


In [103]:
hygiene_df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,inspection_id,inspection_date,inspection_type,violation_id,...,inspection_score,business_latitude,business_longitude,business_location,:@computed_region_fyvs_ahh9,:@computed_region_p5aj_wyqh,:@computed_region_rxqg_mtj9,:@computed_region_yftq_j783,:@computed_region_bh8s_q3mv,:@computed_region_ajp5_b2md
0,69618,Fancy Wheatfield Bakery,1362 Stockton St,San Francisco,CA,94133,69618_20190304,2019-03-04T00:00:00.000,Complaint,69618_20190304_103130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,97975,BREADBELLY,1408 Clement St,San Francisco,CA,94118,97975_20190725,2019-07-25T00:00:00.000,Routine - Unscheduled,97975_20190725_103124,...,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69487,Hakkasan San Francisco,1 Kearny St,San Francisco,CA,94108,69487_20180418,2018-04-18T00:00:00.000,Routine - Unscheduled,69487_20180418_103119,...,88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91044,Chopsticks Restaurant,4615 Mission St,San Francisco,CA,94112,91044_20170818,2017-08-18T00:00:00.000,Non-inspection site visit,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,85987,Tselogs,552 Jones St,San Francisco,CA,94102,85987_20180412,2018-04-12T00:00:00.000,Routine - Unscheduled,85987_20180412_103132,...,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will now use the Foursquare API to search each neighborhoods restaurants

In [174]:
client_id = 'U0BHFR2CGBOER0NS2E3LDULEVT032SXA3KVWLR2U1RTQBJCV' # your Foursquare ID
client_secret = 'WRRQIHUGH45BSIKD4HCNE5ZXRNAK3E1JJNIXVNRVBNYLZYEC' # your Foursquare Secret
version = '20180605' # Foursquare API version
category= '4d4b7105d754a06374d81259' #Food Category
limit=1000


print('Your credentails:')
print('CLIENT_ID: ' + client_id)
print('CLIENT_SECRET:' + client_secret)

Your credentails:
CLIENT_ID: U0BHFR2CGBOER0NS2E3LDULEVT032SXA3KVWLR2U1RTQBJCV
CLIENT_SECRET:WRRQIHUGH45BSIKD4HCNE5ZXRNAK3E1JJNIXVNRVBNYLZYEC


In [253]:
def getVenuesLoc(names, radius=600):
    
    venues_list=[]
    unexplored_nhoods=[]
    explored_nhoods=[]
    for name in names:
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&near={},San Francisco, CA&categoryId={}&radius={}&limit={}'.format(
        client_id,
        client_secret,
        version,
        name,
        category,
        radius, 
        limit)
            
        # make the GET request for neighbourhoods that don't throw error
        results = requests.get(url).json()
        if 'errorType' in results['meta']:
            print("Couldn't get venues from:", name)
            unexplored_nhoods.append(name)
        else:
            print(name)
            explored_nhoods.append(name)
            results = results["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
            venues_list.append([(
            name,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = ['Neighbourhood', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues, explored_nhoods, unexplored_nhoods)

Becuase there are certain neighborhoods grouped, we ungroup them to apply the function

In [254]:
nhood_names=[]
for name in nhoods_df['nbrhood']:
    if '/' in name:
        split_name=name.split('/',1)
        nhood_names.append(split_name[0].strip())
        nhood_names.append(split_name[1].strip())
    elif '/' not in name:
        nhood_names.append(name)
    

In [255]:
sf_venues, explored_nhoods, unexplored_nhoods = getVenuesLoc(nhood_names)

Alamo Square
Anza Vista
Balboa Terrace
Couldn't get venues from: Bayview
Bernal Heights
Buena Vista Park
Ashbury Heights
Couldn't get venues from: Central Richmond
Central Sunset
Clarendon Heights
Couldn't get venues from: Corona Heights
Cow Hollow
Crocker Amazon
Couldn't get venues from: Diamond Heights
Downtown
Duboce Triangle
Couldn't get venues from: Eureka Valley
Couldn't get venues from: Dolores Heights
Excelsior
Financial District
Couldn't get venues from: Barbary Coast
Couldn't get venues from: Yerba Buena
Forest Hill
Couldn't get venues from: Forest Hills Extension
Forest Knolls
Glen Park
Golden Gate Heights
Golden Gate Park
Haight Ashbury
Hayes Valley
Hunters Point
Ingleside
Ingleside Heights
Ingleside Terrace
Couldn't get venues from: Inner Mission
Inner Parkside
Couldn't get venues from: Inner Richmond
Inner Sunset
Jordan Park
Laurel Heights
Couldn't get venues from: Lake Street
Monterey Heights
Couldn't get venues from: Lake Shore
Lakeside
Lone Mountain
Lower Pacific Heigh

We now need to check our hyegiene dataframe in order to update the hygiene score and status of each of the restaurants we've gathered from the Foursquare API. Before that, we clean up the hygiene_df, keeping only the columns we need and the rows that contain values.

In [268]:
hygiene_df.columns

Index(['business_id', 'business_name', 'business_address', 'business_city',
       'business_state', 'business_postal_code', 'inspection_id',
       'inspection_date', 'inspection_type', 'violation_id',
       'violation_description', 'risk_category', 'business_phone_number',
       'inspection_score', 'business_latitude', 'business_longitude',
       'business_location', ':@computed_region_fyvs_ahh9',
       ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9',
       ':@computed_region_yftq_j783', ':@computed_region_bh8s_q3mv',
       ':@computed_region_ajp5_b2md'],
      dtype='object')

In [271]:
hygiene_df=hygiene_df[['business_name','business_address','inspection_date', 'inspection_type','violation_description', 'risk_category','inspection_score' ]]
hygiene_df = hygiene_df[hygiene_df['inspection_score'].notna()]
hygiene_df

,business_name,business_address,inspection_date,inspection_type,violation_description,risk_category,inspection_score
1,BREADBELLY,1408 Clement St,2019-07-25T00:00:00.000,Routine - Unscheduled,Inadequately cleaned or sanitized food contact...,Moderate Risk,96
2,Hakkasan San Francisco,1 Kearny St,2018-04-18T00:00:00.000,Routine - Unscheduled,Inadequate and inaccessible handwashing facili...,Moderate Risk,88
4,Tselogs,552 Jones St,2018-04-12T00:00:00.000,Routine - Unscheduled,Improper thawing methods,Moderate Risk,94
8,"The Estate Kitchen, LLC",799 Bryant St,2018-04-16T00:00:00.000,Routine - Unscheduled,Improper food storage,Low Risk,86
9,Beloved Cafe,3338 24th St,2018-05-02T00:00:00.000,Routine - Unscheduled,Low risk vermin infestation,Low Risk,96
...,...,...,...,...,...,...,...
53967,El Gran Taco Loco,4591 Mission St.,2019-05-06T00:00:00.000,Routine - Unscheduled,Insufficient hot water or running water,Moderate Risk,76
53968,Blue Bottle Coffee,2 South Park,2019-05-06T00:00:00.000,Routine - Unscheduled,Inadequately cleaned or sanitized food contact...,Moderate Risk,80
53970,Philz Coffee,300 Folsom St,2019-05-06T00:00:00.000,Routine - Unscheduled,Foods not protected from contamination,Moderate Risk,92
53971,El Gran Taco Loco,4591 Mission St.,2019-05-06T00:00:00.000,Routine - Unscheduled,Inadequate food safety knowledge or lack of ce...,Moderate Risk,76


Now we can Map out the locations of each restaurant!

In [277]:
hygiene_df.sort_values('inspection_date', ascending=False).drop_duplicates(subset='business_name', keep='first')

,business_name,business_address,inspection_date,inspection_type,violation_description,risk_category,inspection_score
14552,Frisco Fried,5176 03rd St,2019-10-03T00:00:00.000,Routine - Unscheduled,Low risk vermin infestation,Low Risk,92
6787,Cafe Majestic,1500 SUTTER St,2019-10-03T00:00:00.000,Routine - Unscheduled,Low risk vermin infestation,Low Risk,84
13344,Sears Fine Food,439 Powell St,2019-10-03T00:00:00.000,Routine - Unscheduled,Unapproved or unmaintained equipment or utensils,Low Risk,91
11908,Tokyo Express,160 Spear St Lobby ID,2019-10-03T00:00:00.000,Routine - Unscheduled,Foods not protected from contamination,Moderate Risk,87
14354,SHERIDAN ELEMENTARY SCHOOL,431 CAPITOL Ave,2019-10-03T00:00:00.000,Routine - Unscheduled,Inadequate food safety knowledge or lack of ce...,Moderate Risk,92
...,...,...,...,...,...,...,...
25331,Sally's Restaurant and Deli,300 De Haro St #332,2016-10-06T00:00:00.000,Routine - Unscheduled,Unclean or degraded floors walls or ceilings,Low Risk,71
27998,CATER THYME,1 UNITED NATIONS Plz,2016-10-05T00:00:00.000,Routine - Unscheduled,NaN,NaN,100
24684,Way To Life Foods,1 United Nations Plaza,2016-10-05T00:00:00.000,Routine - Unscheduled,NaN,NaN,100
24270,Hey Hey Gourmet,1 United Nations Plaza,2016-10-05T00:00:00.000,Routine - Unscheduled,NaN,NaN,100


In [278]:
sf_venues['inspection_score'] = sf_venues['Venue'].map(hygiene_df.set_index('business_name')['inspection_score'])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [263]:
sf_center = [37.7749, -122.4194]
sf_map = folium.Map(location=sf_center, zoom_start=13)
folium.Marker(sf_center, popup='City Center').add_to(sf_map)
for name, lat, lng in zip(sf_venues.Venue, sf_venues.Venue_Latitude, sf_venues.Venue_Longitude):
    color = 'blue'
    folium.CircleMarker([lat, lng], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(sf_map)
sf_map